In [11]:
import ONS_scraper_functions as osf
import pandas as pd

# Allow us to see more of each column
pd.options.display.max_colwidth = 250

In [24]:
# Start by fetching a complete list of the titles and a description of ONS datasets (available via the api)
titles, descriptions = osf.get_ONS_datasets_titles_descriptions()

# For more easy visualisation the lists are placed in a dataframe
ONS_api_datasets_df = pd.DataFrame({'Titles': titles[:5], 'Description': descriptions[:5]})

# Visualise the first 5 rows of the dataframe
ONS_api_datasets_df.head()

,Titles,Description
0,Quarterly personal well-being estimates,"Seasonally and non seasonally-adjusted quarterly estimates of life satisfaction, feeling that the things done in life are worthwhile, happiness and anxiety in the UK."
1,Personal well-being estimates by local authority,"Estimates of life satisfaction, feeling that the things done in life are worthwhile, happiness and anxiety at the UK, country, regional, county, local and unitary authority level."
2,Deaths registered weekly in England and Wales by region,"Provisional counts of the number of deaths registered in England and Wales, by region, in the latest weeks for which data are available."
3,Death registrations and occurrences by local authority and place of death,"Provisional counts of the number of deaths registered in England and Wales, including deaths involving the coronavirus (COVID-19), by local authority, health board and place of death in the latest weeks for which data are available."
4,Death registrations and occurrences by health board and place of death,"Provisional counts of the number of deaths registered in Wales, by health board and place of death, in the latest weeks for which data are available."


In [16]:
""" 
Before we can get the column titles we need the relevant urls.
We can get a list of a urls for each available dataset by simply using the get_ONS_datasets_urls function.
The urls this function collect aren't a direct download link, however, they do provide useful information.
They essentially just provide some metadata such as release date, state, version, and various download 
links for different file types. 
"""
urls = osf.get_ONS_datasets_urls()

# The URLs for the first 5 datasets in the api
pd.DataFrame(urls, columns=['URLs']).head()

,URLs
0,https://api.beta.ons.gov.uk/v1/datasets/wellbeing-quarterly/editions/time-series/versions/6
1,https://api.beta.ons.gov.uk/v1/datasets/wellbeing-local-authority/editions/time-series/versions/3
2,https://api.beta.ons.gov.uk/v1/datasets/weekly-deaths-region/editions/covid-19/versions/133
3,https://api.beta.ons.gov.uk/v1/datasets/weekly-deaths-local-authority/editions/2022/versions/65
4,https://api.beta.ons.gov.uk/v1/datasets/weekly-deaths-health-board/editions/2022/versions/65


In [19]:
# Now we have the relevant URLs we can find the column titles for each dataset
col_titles_example = osf.find_ONS_cols(urls[0])
col_titles_example

['v4_2',
 'LCL',
 'UCL',
 'yyyy-qq',
 'Time',
 'uk-only',
 'Geography',
 'measure-of-wellbeing',
 'MeasureOfWellbeing',
 'wellbeing-estimate',
 'Estimate',
 'seasonal-adjustment',
 'SeasonalAdjustment']

In [21]:
"""
The function can only take a single URL at a time and therefore if we want to get the column titles
for every dataset we must loop through the urls.
This function works by downloading the dataset and reading the column titles which can be quite slow
depending on download speeds so its recommended to avoid looping through every dataset unless 
necessary. This is the reason we only loop through 5 of the urls.
"""
column_titles = []
count = 0
for url in urls:
    if count > 4:
        break

    column_titles.append(osf.find_ONS_cols(url))
    count += 1

In [25]:
# Updating the dataframe to include column titles
ONS_api_datasets_df['Column_Titles'] = column_titles
ONS_api_datasets_df.head()

,Titles,Description,Column_Titles
0,Quarterly personal well-being estimates,"Seasonally and non seasonally-adjusted quarterly estimates of life satisfaction, feeling that the things done in life are worthwhile, happiness and anxiety in the UK.","[v4_2, LCL, UCL, yyyy-qq, Time, uk-only, Geography, measure-of-wellbeing, MeasureOfWellbeing, wellbeing-estimate, Estimate, seasonal-adjustment, SeasonalAdjustment]"
1,Personal well-being estimates by local authority,"Estimates of life satisfaction, feeling that the things done in life are worthwhile, happiness and anxiety at the UK, country, regional, county, local and unitary authority level.","[v4_3, Data marking, Lower limit, Upper limit, yyyy-yy, Time, administrative-geography, Geography, measure-of-wellbeing, MeasureOfWellbeing, wellbeing-estimate, Estimate]"
2,Deaths registered weekly in England and Wales by region,"Provisional counts of the number of deaths registered in England and Wales, by region, in the latest weeks for which data are available.","[v4_1, Data Marking, calendar-years, Time, administrative-geography, Geography, week-number, Week, recorded-deaths, Deaths]"
3,Death registrations and occurrences by local authority and place of death,"Provisional counts of the number of deaths registered in England and Wales, including deaths involving the coronavirus (COVID-19), by local authority, health board and place of death in the latest weeks for which data are available.","[v4_0, calendar-years, Time, administrative-geography, Geography, week-number, Week, cause-of-death, CauseOfDeath, place-of-death, PlaceOfDeath, registration-or-occurrence, RegistrationOrOccurrence]"
4,Death registrations and occurrences by health board and place of death,"Provisional counts of the number of deaths registered in Wales, by health board and place of death, in the latest weeks for which data are available.","[v4_0, calendar-years, Time, local-health-board, Geography, week-number, Week, cause-of-death, CauseOfDeath, place-of-death, PlaceOfDeath, registration-or-occurrence, RegistrationOrOccurrence]"


In [80]:
"""
Example of how the data collected can be used

Further data from any of the available datasets could be downloaded and used in the same way the column
titles were
"""

import plotly.express as px
import seaborn as sns
color_scale = list(reversed(list(sns.color_palette('flare').as_hex())))

col_title_freq = ONS_api_datasets_df['Column_Titles'].explode().value_counts()

fig = px.bar(
    col_title_freq[col_title_freq>1],
    color='value',
    color_continuous_scale=color_scale,
    width=700,
    height=700)

fig.update_layout(font=dict(family='calibri', size=16), 
                  xaxis=dict(title='Column Title', title_font_size=22), 
                  yaxis=dict(title='Frequency', title_font_size=22, showgrid=True, gridcolor='lightgrey'),
                  title=dict(text="Column Titles That Appear in Multiple Datasets", font=dict(size=30)),
                  coloraxis_showscale=False,
                  plot_bgcolor = "rgba(255,255,255,1)")

fig.show()